In [ ]:
!pip install snowflake-connector-python

In [ ]:
import pandas as pd
import snowflake.connector

conn=snowflake.connector.connect(
    user='PadmapriyaS',
    password='Padmaat2006priya',
    account='EVFQUEB-BZ93641',
    database='SURVEY',
    schema='PUBLIC',
    warehouse='COMPUTE_WH'
)

In [ ]:
#Run query to fetch data

query='select * from survey_lung_cancer'
df=pd.read_sql(query,conn)
conn.close()
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
x=df.drop('LUNG_CANCER',axis=1)
y=df['LUNG_CANCER']

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
# Label Encoding (TARGET)
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
y_train = le_y.fit_transform(y_train)
y_test = le_y.transform(y_test)

# Label Encoding (GENDER only)
le_gender = LabelEncoder()
x_train['GENDER'] = le_gender.fit_transform(x_train['GENDER'])
x_test['GENDER'] = le_gender.transform(x_test['GENDER'])


In [ ]:
#Apply SMOTE (TRAIN DATA ONLY)

from imblearn.over_sampling import SMOTE
smote=SMOTE(random_state=42)
x_train_smote,y_train_smote=smote.fit_resample(x_train,y_train)

In [ ]:
import pandas as pd
pd.Series(y_train_smote).value_counts()

In [ ]:
#Feature Scaling (AFTER SMOTE)
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train_smote=sc.fit_transform(x_train_smote)
x_test=sc.transform(x_test)

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb_pipeline = Pipeline(steps=[
    ('smote', SMOTE(random_state=42)),
    ('model', GaussianNB())
])


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

nb_scores = cross_val_score(
    nb_pipeline,
    x_train,
    y_train,
    cv=skf,
    scoring='f1'
)

print("Naive Bayes F1 scores:", nb_scores)
print("Mean NB F1:", nb_scores.mean())


In [ ]:
# Fit Naive Bayes
nb_pipeline.fit(x_train, y_train)
y_pred_nb = nb_pipeline.predict(x_test)
y_prob_nb = nb_pipeline.predict_proba(x_test)[:, 1]


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [ ]:
cm_nb = confusion_matrix(y_test, y_pred_nb)
disp_nb = ConfusionMatrixDisplay(confusion_matrix=cm_nb)

disp_nb.plot()
plt.title("Naive Bayes Confusion Matrix")
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc


In [ ]:
fpr_nb, tpr_nb, _ = roc_curve(y_test, y_prob_nb)
roc_auc_nb = auc(fpr_nb, tpr_nb)

plt.figure()
plt.plot(fpr_nb, tpr_nb, label=f"NB AUC = {roc_auc_nb:.2f}")
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Naive Bayes ROC Curve")
plt.legend()
plt.show()
